In [20]:
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
import scipy.cluster.vq as scv
import pickle

# os.system("cd $CAFFE_ROOT")
caffe_root = os.environ["CAFFE_ROOT"]
os.chdir(caffe_root)
print caffe_root
sys.path.insert(0, caffe_root + 'python')
import caffe

caffe.set_mode_gpu()
caffe.set_device(2)
option = 'lenet5'
if option == 'lenet5':
    prototxt = '3_prototxt_solver/lenet5/train_val.prototxt'             
    caffemodel = '4_model_checkpoint/lenet5/lenet5.caffemodel'
    iters = 100
    dir_t = '2_results/kmeans/lenet5/'
elif option == 'alexnet':
    prototxt = '3_prototxt_solver/L2/train_val.prototxt'             
    caffemodel = '4_model_checkpoint/alexnet/alexnet9x.caffemodel'  
    iters = 1000
    dir_t = '2_results/kmeans/alexnet/'
elif option == 'vgg':
    prototxt = '3_prototxt_solver/vgg16/train_val.prototxt'             
    caffemodel = '4_model_checkpoint/vgg16/vgg16_12x.caffemodel'  
    iters = 1000
    dir_t = '2_results/kmeans/vgg16/'

log = dir_t + 'log_accu'

/home/songhan/pruning/


In [21]:
choice = [64,16]
net = caffe.Net(prototxt, caffemodel, caffe.TRAIN)

layers = ["conv1", "conv2", "ip1", "ip2"]
num_c = [8, 8, 8, 8]

# layers = ["ip2"]
# num_c =[4]
print "layers TBD: ", layers
print "num_c = ", num_c

layers TBD:  ['conv1', 'conv2', 'ip1', 'ip2']
num_c =  [8, 8, 8, 8]


In [23]:
print "==============1 Perform K-means============="
codebook = {}
for idx, layer in enumerate(layers):
    print "Eval layer:", layer
    W = net.params[layer][0].data.flatten()
    W = W[np.where(W != 0)]
    std = np.std(W)
    initial_uni = np.linspace(-4 * std, 4 * std, num_c[idx]-1)
    codebook[layer],_= scv.kmeans(W, initial_uni)    
    codebook[layer] = np.append(0.0, codebook[layer])
    print "codebook:", codebook[layer]
    print "codebook size:", len(codebook[layer])

==============1 Perform K-means=============
Eval layer: conv1
codebook: [ 0.         -0.29499978 -0.03177995  0.25980002  0.51812828]
codebook size: 5
Eval layer: conv2
codebook: [ 0.         -0.14003672 -0.08655009 -0.04095863  0.05826917  0.12395576
  0.21972357]
codebook size: 7
Eval layer: ip1
codebook: [ 0.         -0.07194122 -0.04054011 -0.01950156  0.01798702  0.03304647
  0.05246379  0.08222321]
codebook size: 8
Eval layer: ip2
codebook: [ 0.         -0.25988275 -0.17693673 -0.10709237  0.13464746  0.23517904]
codebook size: 6


In [43]:
print "================2 Perform quantization=============="
codeDict={}
maskCode={}
for layer in layers:
    print "Quantize layer:", layer
    W = net.params[layer][0].data
    codes, dist = scv.vq(W.flatten(), codebook[layer])
    W_q = np.reshape(codebook[layer][codes], W.shape)
    net.params[layer][0].data[...] = W_q

    maskCode[layer] = np.reshape(codes, W.shape)
    codeBookSize = len(codebook[layer])    
    print "W_q.shape=", W_q.shape        
    print "codebook length=", codeBookSize
    print "maskcode:", maskCode[layer].flatten().shape
    print "maskcode:", np.flatnonzero(maskCode[layer]).shape
    a = maskCode[layer].flatten()
    b = xrange(len(a))
#     print a
#     print b

    codeDict[layer]={}
    for i in xrange(len(a)):
        codeDict[layer].setdefault(a[i], []).append(b[i])
#     print "codeDict  is",codeDict
#     print maskCode[layer]
    

 ================2 Perform quantization==============
Quantize layer: conv1
W_q.shape= (20, 1, 5, 5)
codebook length= 5
maskcode: (500,)
maskcode: (280,)
Quantize layer: conv2
W_q.shape= (50, 20, 5, 5)
codebook length= 7
maskcode: (25000,)
maskcode: (2728,)
Quantize layer: ip1
W_q.shape= (500, 800)
codebook length= 8
maskcode: (400000,)
maskcode: (30875,)
Quantize layer: ip2
W_q.shape= (10, 500)
codebook length= 6
maskcode: (5000,)
maskcode: (958,)


In [41]:
print "================3 Perform fintuning=============="
print codebook
lr=0.00001
import time
start_time=time.time()
for i in xrange(1000):
    net.forward()
    net.backward()
    for layer in layers:
#         print layer
        diff=net.params[layer][0].diff.flatten()
        W1 =  net.params[layer][0].data
#         print W1.shape
        codeBookSize=len(codebook[layer])
#         print codeDict[layer]
        for code in xrange(codeBookSize):
            if code==0: continue;
            indexes = codeDict[layer][code]
#             print codeDict[layer]
#           print "diff array is ",diff[indexes] #if one want to go up, other want to go down, that's not good
            codebook[layer][code] -= lr*sum(diff[indexes])/len(diff[indexes])
#             print lr*sum(diff[indexes])/len(diff[indexes])
        W2 = codebook[layer][maskCode[layer]]

#         if lr==0:
#             assert ((W1==W2).all())
        
        net.params[layer][0].data[...]=W2
print time.time()-start_time        
#     if i%200==0:
#         print codebook["ip2"]
    


================3 Perform fintuning==============
{'ip2': array([ 0.        , -0.30827944, -0.22033117, -0.12869972,  0.16594969,
        0.28270735]), 'conv2': array([ 0.        , -0.20370411, -0.13133167, -0.07489873,  0.03206986,
        0.0953579 ,  0.16920747]), 'conv1': array([ 0.        , -0.42127984, -0.08663357,  0.14976108,  0.37965974]), 'ip1': array([ 0.        , -0.11421262, -0.06721763, -0.03180971,  0.00937421,
        0.02238155,  0.03940598,  0.06676778])}
13.0099232197


In [215]:
print codebook["ip2"]

[ 0.         -0.16629212  0.13232081  0.23106943]


In [38]:
print "batch size=",net.blobs['label'].data.shape

import time
start_time = time.time()
for i in xrange(1000):
    net.forward()
    net.backward()
    for layer in layers:        
        diff=net.params[layer][0].diff
        W=    net.params[layer][0].data
        W -= 0.000001*diff    
        net.params[layer][0].data[...]=W
        
print time.time()-start_time
correct = 0
for test_it in range(50000/64):
    net.forward()
    correct += sum(net.blobs['ip2'].data.argmax(1)
                   == net.blobs['label'].data)
print correct / float(50000)

batch size= (64,)
1.99893403053
0.9639


In [29]:
correct = 0
for test_it in range(50000/64):
    net.forward()
    correct += sum(net.blobs['ip2'].data.argmax(1)
                   == net.blobs['label'].data)
print correct / float(50000)

0.9955


0.9984
 

In [39]:
print codebook

{'ip2': array([ 0.        , -0.25988275, -0.17693673, -0.10709237,  0.13464746,
        0.23517904]), 'conv2': array([ 0.        , -0.14003672, -0.08655009, -0.04095863,  0.05826917,
        0.12395576,  0.21972357]), 'conv1': array([ 0.        , -0.29499978, -0.03177995,  0.25980002,  0.51812828]), 'ip1': array([ 0.        , -0.07194122, -0.04054011, -0.01950156,  0.01798702,
        0.03304647,  0.05246379,  0.08222321])}
